# ATLAS Open-Data Statistics Tutorial: Making a likelihood model

This is the first part of the statistics tutorial using ATLAS open data:
1) Using open-data to run an analysis, run background function fits, then make a likelihood model
2) Re-using public likelihood models

The beginning part will follow closely the existing notebook to gather Higgs to diphoton data: https://github.com/atlas-outreach-data-tools/notebooks-collection-opendata/blob/master/13-TeV-examples/uproot_python/HyyAnalysis.ipynb
You can find more information on how to access the ATLAS open-dataset in that tutroial

You can proceed directly to the second section, but still run these cells to collect the data

## Run the Higgs->diphoton analysis

This is almost directly copied from the previous notebook:
https://github.com/atlas-outreach-data-tools/notebooks-collection-opendata/blob/master/13-TeV-examples/uproot_python/HyyAnalysis.ipynb

In [1]:
import sys
import uproot # for reading .root files
import time # to measure time to analyse
import math # for mathematical functions such as square root
import awkward as ak # for handling complex and nested data structures efficiently
import numpy as np # # for numerical calculations such as histogramming
import vector #to use vectors

In [2]:
import atlasopenmagic as atom
atom.set_release('2025e-13tev-beta')

Fetching and caching all metadata for release: 2025e-13tev-beta...
Fetched 374 datasets so far...
Successfully cached 374 datasets.
Active release: 2025e-13tev-beta. (Datasets path: REMOTE)


In [3]:
# Select the skim to use for the analysis
skim = "GamGam"

# Let's get the list of files to go through
# Notice that we use "cache" so that the files are downloaded locally and not streamed
files_list = atom.get_urls('data', skim, protocol='https', cache=True)

In [4]:
# Cut on the photon reconstruction quality
def cut_photon_reconstruction(photon_isTightID):
    # Only the events which have True for both photons are kept
    return (photon_isTightID[:,0]==True) & (photon_isTightID[:,1]==True)

# Cut on the transverse momentum
def cut_photon_pt(photon_pt):
# Only the events where photon_pt[0] > 50 GeV and photon_pt[1] > 30 GeV are kept
    return (photon_pt[:,0] > 50) & (photon_pt[:,1] > 30)

# Cut on the energy isolation
def cut_isolation_pt(photon_ptcone20, photon_pt):
# Only the events where the calorimeter isolation is less than 5.5% are kept
    return ((photon_ptcone20[:,0]/photon_pt[:,0]) < 0.055) & ((photon_ptcone20[:,1]/photon_pt[:,1]) < 0.055)

# Cut on the pseudorapidity in barrel/end-cap transition region
def cut_photon_eta_transition(photon_eta):
# Only the events where modulus of photon_eta is outside the range 1.37 to 1.52 are kept
    condition_0 = (np.abs(photon_eta[:, 0]) < 1.52) | (np.abs(photon_eta[:, 0]) > 1.37)
    condition_1 = (np.abs(photon_eta[:, 1]) < 1.52) | (np.abs(photon_eta[:, 1]) > 1.37)
    return condition_0 & condition_1

# This function calculates the invariant mass of the 2-photon state
def calc_mass(photon_pt, photon_eta, photon_phi, photon_e):
    p4 = vector.zip({"pt": photon_pt, "eta": photon_eta, "phi": photon_phi, "e": photon_e})
    invariant_mass = (p4[:, 0] + p4[:, 1]).M # .M calculates the invariant mass
    return invariant_mass

# Cut on null diphoton invariant mass
def cut_mass(invariant_mass):
    return (invariant_mass != 0)

# Cut on the pT relative to the invariant mass
# Only the events where the invididual photon pT is larger than 35% of the invariant mass are kept
def cut_iso_mass(photon_pt, invariant_mass):
    return ((photon_pt[:,0]/invariant_mass) > 0.35) & ((photon_pt[:,1]/invariant_mass) > 0.35)

In [5]:
# Controls the fraction of all the events analysed. All of the data is used by
# default to run this analysis (implemented in the loop over the tree). Reduce
# this if you want the code to run quicker. This can take several minutes to
# process the entire dataset
fraction = 1.0

# Holder for the masses as we process the files
sample_data = []
variables = ["photon_pt","photon_eta","photon_phi","photon_e","photon_isTightID","photon_ptcone20"]

# Loop over all the files in our list
for afile in files_list:

    # Print which sample is being processed
    print(f'Processing file {afile} ({files_list.index(afile)}/{len(files_list)})')

    # Open file
    tree = uproot.open(afile + ":analysis")

    numevents = tree.num_entries

    # Perform the cuts for each data entry in the tree and calculate the invariant mass
    for data in tree.iterate(variables, library="ak", entry_stop=int(numevents*fraction)):

        photon_isTightID = data['photon_isTightID']
        data = data[cut_photon_reconstruction(photon_isTightID)]

        photon_pt = data['photon_pt']
        data = data[cut_photon_pt(photon_pt)]

        data = data[cut_isolation_pt(data['photon_ptcone20'],data['photon_pt'])]

        photon_eta = data['photon_eta']
        data = data[cut_photon_eta_transition(photon_eta)]

        data['mass'] = calc_mass(data['photon_pt'], data['photon_eta'], data['photon_phi'], data['photon_e'])

        data = data[cut_mass(data['mass'])]

        data = data[cut_iso_mass(data['photon_pt'], data['mass'])]

        # Append data to the whole sample data list
        sample_data.append(data['mass'])
        
# turns sample_data back into an awkward array
all_data = ak.concatenate(sample_data)

Processing file simplecache::https://opendata.cern.ch/eos/opendata/atlas/rucio/opendata/ODEO_FEB2025_v0_GamGam_data15_periodD.GamGam.root (0/16)
Processing file simplecache::https://opendata.cern.ch/eos/opendata/atlas/rucio/opendata/ODEO_FEB2025_v0_GamGam_data15_periodE.GamGam.root (1/16)
Processing file simplecache::https://opendata.cern.ch/eos/opendata/atlas/rucio/opendata/ODEO_FEB2025_v0_GamGam_data15_periodF.GamGam.root (2/16)
Processing file simplecache::https://opendata.cern.ch/eos/opendata/atlas/rucio/opendata/ODEO_FEB2025_v0_GamGam_data15_periodG.GamGam.root (3/16)
Processing file simplecache::https://opendata.cern.ch/eos/opendata/atlas/rucio/opendata/ODEO_FEB2025_v0_GamGam_data15_periodH.GamGam.root (4/16)
Processing file simplecache::https://opendata.cern.ch/eos/opendata/atlas/rucio/opendata/ODEO_FEB2025_v0_GamGam_data15_periodJ.GamGam.root (5/16)
Processing file simplecache::https://opendata.cern.ch/eos/opendata/atlas/rucio/opendata/ODEO_FEB2025_v0_GamGam_data16_periodA.GamG

FSTimeoutError: 

## Estimating a background

Okay now we have data! Unfortunately though the signal we want to see isn't the only thing there: there is also some backgrounds. In general there are 2 ways we evaluate the backgrounds in high-energy experiments:
1) Through dedicated simulations for each of the potential background proceses, which we apply the same analysis selection to evaluate the number of backgrounds
2) By evaluating the background directly in the data

For the Higgs diphoton analysis option 2) was used. But if we are *estimating something from data*, this is a **statistical method!**. So let's get started!

First let's import some module for plotting, and we will use `lmfit` to do the fitting later

In [ ]:
import matplotlib.pyplot as plt # for plotting
from matplotlib.ticker import MaxNLocator,AutoMinorLocator # for minor ticks
import lmfit # for the signal and background fits

Now let's make a first plot of the data

In [ ]:
# x-axis range of the plot
xmin = 100 #GeV
xmax = 160 #GeV

# Histogram bin setup
step_size = 1 #GeV
bin_edges = np.arange(start=xmin, # The interval includes this value
                    stop=xmax+step_size, # The interval doesn't include this value
                    step=step_size ) # Spacing between values
bin_centres = np.arange(start=xmin+step_size/2, # The interval includes this value
                        stop=xmax+step_size/2, # The interval doesn't include this value
                        step=step_size ) # Spacing between values

# Creating histogram from data
data_x,_ = np.histogram(ak.to_numpy(all_data),
                        bins=bin_edges ) # histogram the data
data_x_errors = np.sqrt( data_x ) # statistical error on the data


# *************
# Main plot
# *************
main_axes = plt.gca() # get current axes

# plot the data points
main_axes.errorbar(x=bin_centres, y=data_x, yerr=data_x_errors,
                    fmt='ko', # 'k' means black and 'o' is for circles
                    label='Data')

# set the x-limit of the main axes
main_axes.set_xlim( left=xmin, right=xmax )

# separation of x axis minor ticks
main_axes.xaxis.set_minor_locator( AutoMinorLocator() )

# set the axis tick parameters for the main axes
main_axes.tick_params(which='both', # ticks on both x and y axes
                        direction='in', # Put ticks inside and outside the axes
                        top=True, # draw ticks on the top axis
                        right=True ) # draw ticks on right axis

# x-axis label
main_axes.set_xlabel(r'di-photon invariant mass $\mathrm{m_{\gamma\gamma}}$ [GeV]',
                    fontsize=13, x=1, horizontalalignment='right' )

# write y-axis label for main axes
main_axes.set_ylabel('Events / '+str(step_size)+' GeV',
                        y=1, horizontalalignment='right')

# set y-axis limits for main axes
main_axes.set_ylim( bottom=0, top=np.amax(data_x)*1.6 )

# add minor ticks on y-axis for main axes
main_axes.yaxis.set_minor_locator( AutoMinorLocator() )

# draw the legend
main_axes.legend( frameon=False ); # no box around the legend

Indeed, it's not obvious there is some signal here, and we have lots of background we need to estimate first

So let's make a model to estimate our background and signal. We will do this via *least squares* fitting. For this we will need to define some functions we think our data will follow. We'll model our:
- Background: As a polynomial function $p(x)=a_0+a_1 x+ a_2 x^2+ ...$
- Signal: As a Gaussian peak $g(x)=Gauss(\mu,\sigma)$ with mass $\mu$ and width $\sigma$
We'll treat our total model as a sum of these two functions multiplied by the number of events of each: $f(x)=N_{background}p(x)+N_{signal}g(x)$

We will fix our mass to some specific value, specify the order of the polynomial then we will fit simultaneously all the background parameters, and the number of signal/background events by minimzing the least squares: $$\sum_i \frac{(y_i-f(x_i))^2}{\sigma^2_i} $$
where $x_i, y_i$ are the fitted data coordinates with an uncertainity $\sigma_i$, and the fitted function is $f(x_i)$

While this sounds complicated, there are many existing packages which run least square fits, and so after specifying the model, it is almost completely automated. For convenience we will actually define this as a function so we can play with things

In [ ]:
def fitModel(bin_centres, data_x, mass=125, degree=4):
    
    # Define the signal and background mopel
    polynomial_mod = lmfit.models.PolynomialModel( degree ) # 4th order polynomial
    gaussian_mod = lmfit.models.GaussianModel() # Gaussian

    # set initial guesses for the parameters of the polynomial model
    pars = polynomial_mod.guess(data_x, x=bin_centres) #use a pre-guess from the data

    # set initial guesses and ranges for the parameters of the Gaussian model
    pars += gaussian_mod.make_params(center=dict(value=mass, vary=False),
                       amplitude=dict(value=100, min=0, max=1e4),
                       sigma=dict(value=2, min=1, max=5))

    # Combined model
    model = polynomial_mod + gaussian_mod

    # fit the model to the data, this is most of the work!
    result = model.fit(data_x, # data to be fit
                    pars, # guesses for the parameters
                    x=bin_centres, weights=1/data_x_errors ) 

    
    # This stores the result of the fit
    params_dict = result.params # get the parameters from the fit to data

    # background part of fit
    background_x=polynomial_mod.eval(params=params_dict,x=bin_centres)

    # signal part of the fit
    signal_x = gaussian_mod.eval(params=params_dict,x=bin_centres)

    # get the uncertainity of the fit
    model_x_errors = result.eval_uncertainty(sigma=1)
    
    return signal_x, background_x, model_x_errors, result

We will also make a function to plot these results, this is the similar to the original notebook

In [ ]:
def plotAnalysis(bin_centres, data_x, data_x_errors, background_x, signal_x, model_x_errors):
    # *************
    # Main plot
    # *************
    plt.axes([0.1,0.3,0.85,0.65]) # left, bottom, width, height
    main_axes = plt.gca() # get current axes

    # plot the data points
    main_axes.errorbar(x=bin_centres, y=data_x, yerr=data_x_errors, fmt='ko', label='Data', markersize=4) # black circles

    # plot the background only fit
    main_axes.plot(bin_centres, background_x, '--b', label='Bkg') # dashed blue line

    plt.fill_between(bin_centres, background_x-model_x_errors, background_x+model_x_errors, color="gray", label=r'1-$\sigma$ uncertainty band')

    # plot the signal + background fit
    main_axes.plot(bin_centres, signal_x+background_x,'-r', label='Sig+Bkg') # single red line

    # set the x-limit of the main axes
    main_axes.set_xlim( left=xmin, right=xmax )

    # separation of x-axis minor ticks
    main_axes.xaxis.set_minor_locator( AutoMinorLocator() )

    # set the axis tick parameters for the main axes
    main_axes.tick_params(which='both', # ticks on both x and y axes
                        direction='in', # Put ticks inside and outside the axes
                        top=True, # draw ticks on the top axis
                        labelbottom=False, # don't draw tick labels on bottom axis
                        right=True ) # draw ticks on right axis

    # write y-axis label for main
    main_axes.set_ylabel('Events / '+str(step_size)+' GeV',
                        horizontalalignment='right')

    # set the y-axis limit for the main axes
    main_axes.set_ylim( bottom=0, top=np.amax(data_x)*1.5 )

    # set minor ticks on the y-axis of the main axes
    main_axes.yaxis.set_minor_locator( AutoMinorLocator() )

    # avoid displaying y=0 on the main axes
    main_axes.yaxis.get_major_ticks()[0].set_visible(False)

    # Add text 'ATLAS Open Data' on plot
    plt.text(0.2, 0.92, 'ATLAS Open Data', transform=main_axes.transAxes, fontsize=13 )

    # Add text 'for education' on plot
    plt.text(0.2, 0.86, 'for education', transform=main_axes.transAxes, style='italic',fontsize=8 )

    lumi = 36.1
    lumi_used = str(lumi*fraction) 
    plt.text(0.2, 0.8, r'$\sqrt{s}$=13 TeV,$\int$L dt = '+lumi_used+' fb$^{-1}$', transform=main_axes.transAxes ) 

    # Add a label for the analysis carried out
    plt.text(0.2, 0.74, r'$H \rightarrow \gamma\gamma$', transform=main_axes.transAxes ) 

    # draw the legend
    main_axes.legend(frameon=False, # no box around the legend
                    loc='lower left' ) # legend location

    # *************
    # Data-Bkg plot
    # *************
    plt.axes([0.1,0.1,0.85,0.2]) # left, bottom, width, height
    sub_axes = plt.gca() # get the current axes

    # set the y axis to be symmetric about Data-Background=0
    sub_axes.yaxis.set_major_locator( MaxNLocator(nbins='auto', symmetric=True) )

    # plot Data-Background
    sub_axes.errorbar(x=bin_centres, y=data_x-background_x, yerr=data_x_errors, fmt='ko',markersize=4 ) # black circles

    # draw the background only fit
    sub_axes.plot(bin_centres, background_x-background_x, '--b' )  # dashed blue line

    # draw the fit to data
    sub_axes.plot(bin_centres, signal_x, '-r' ) # single red line

    plt.fill_between(bin_centres, -model_x_errors, model_x_errors, color="gray", label=r'1-$\sigma$ uncertainty band')

    # set the x-axis limits on the sub axes
    sub_axes.set_xlim( left=xmin, right=xmax )

    # separation of x-axis minor ticks
    sub_axes.xaxis.set_minor_locator( AutoMinorLocator() )

    # x-axis label
    sub_axes.set_xlabel(r'Di-photon invariant mass $\mathrm{m_{\gamma\gamma}}$ [GeV]', x=1, horizontalalignment='right',fontsize=13 )

    # set the tick parameters for the sub axes
    sub_axes.tick_params(which='both', direction='in', top=True, right=True ) 

    # separation of y-axis minor ticks
    sub_axes.yaxis.set_minor_locator( AutoMinorLocator() )

    # y-axis label on the sub axes
    sub_axes.set_ylabel( 'Events-Bkg' )

    # Generic features for both plots
    main_axes.yaxis.set_label_coords( -0.09, 1 ) # x,y coordinates of the y-axis label on the main axes
    sub_axes.yaxis.set_label_coords( -0.09, 0.5 ) # x,y coordinates of the y-axis label on the sub axes

Let's histogram the data

In [ ]:
#Bin edges
xmin = 100 #GeV
xmax = 160 #GeV
step_size = 1 #GeV
bin_edges = np.arange(start=xmin, # The interval includes this value
                    stop=xmax+step_size, # The interval doesn't include this value
                    step=step_size ) # Spacing between values
bin_centres = np.arange(start=xmin+step_size/2, # The interval includes this value
                        stop=xmax+step_size/2, # The interval doesn't include this value
                        step=step_size ) # Spacing between values

data_x,_ = np.histogram(ak.to_numpy(all_data), bins=bin_edges ) # histogram the data
data_x_errors = np.sqrt( data_x ) # statistical error on the data

Now let's run the fit!

In [ ]:
signal_x, background_x, model_x_errors, result=fitModel(bin_centres, data_x)
print(result.fit_report()) #print alos lot's of fit information

You can see that lmfit can output alot of interesting statistical information! We'll get back to some of these in a moment

Let's make the quick plot of the result

In [ ]:
plotAnalysis(bin_centres, data_x, data_x_errors, background_x, signal_x, model_x_errors)

Super nice! Here we chose a fourth degree polynomial, which seems to do a good job, but is that the best choice?

We can run a goodness of fit test to evaluate how well we think our fit model matches the data. We can calulate the *reduced chi square* $$\chi^2_\nu=\frac{\chi^2}{\nu}$$ 
where:
$$\chi^2=\sum_i \frac{(y_i-f(x_i))^2}{\sigma^2_i}$$
and $\nu$ = degrees of freedom = number of fitted paramaters - number of fitted data points

Ideally for a good fit you have $\chi_\nu^2\sim1$. Larger/smaller values means you could be under/over-fitting your data

Okay so to test this assumption let's re-run the fit for backgrounds of degrees 2 to 7. The `lmfit` package nicely automatically calculated the $\chi^2$ and $\nu$ values for you, so let's plot this scan

In [ ]:
degrees=range(2,7)
chi2s=np.zeros(len(degrees))
dofs=np.zeros(len(degrees))

for ii,degree in enumerate(degrees):
    signal_x, background_x, model_x_errors, result=fitModel(bin_centres, data_x, degree=degree)
    chi2s[ii]=result.chisqr
    dofs[ii]=result.nfree

fig,ax=plt.subplots()
ax.plot(degrees,chi2s/dofs)
ax.set_ylabel(r'$\chi^2/\nu$')
ax.set_xlabel('Degree of background polynomial')

So indeed a choice of degree 3 or 4 polynomial seems to be a decent enough choice

There are more advanced statistical methods for choosing the minimal most descriptive model (e.x. F-tests), but that's a story for another day

## Make a likelihood model

Okay so now we have a background and signal template for the data. How do we start making actual stating statistical statements on whethere the data **is** truly consitent with a signal and not just background noice.

In ATLAS we almost always run statistical tests using a template profiled likelihood approach (check the lecture for more details). In this approach we need several pieces:
- A histogram of the data for each region
- A histogram of the signal for each region
- A histogram of the background for each region
- Uncertiainites on the signal/background model which can also be histograms, or specific numbers like normalization uncertainities

The core building block is that each bin of the histograms will be treated like a Poisson counting experiment $$Pois(x_i|\mu s_i+b_i)$$
where the $\mu$ is the signal strength which is a core piece of the model. For $\mu=0$ this is a background only model, while $\mu=1$ is our signal+background model. It's so important we call it the paramater of interest (poi).

For our case we have just the one signal region, the data/signal/background histogram, and lmfit also gave us the uncertainity it predicted on the fit

We will make use of the `pyhf` package to build these likelihood models right in python. The contents are specified in a `json` format, so they are relatively human readable. We will also make use of `cabinetry` which is built on-top of `pyhf` and has some more options for visualization and such

In [ ]:
import pyhf, json, cabinetry

Okay so let's make the likelihood model. As before we will write is as a function so we can play around with things later.

In [ ]:
def makePyhfConfig(data_x,signal_x,background_x,model_x_errors):
    #First we'll make a json file specifying the predictions
    model_spec = {
        #Add the region
        "channels": [
            {
                "name": "region",
                "samples": [
                    {
                        # Add the signal to this region
                        "name": "signal",
                        "data": signal_x.tolist(),
                        # parameter of interest which can scale up/down in the fit
                        "modifiers": [
                            {"name": "mu", "type": "normfactor", "data":None}  
                        ]
                    },
                    {
                        # Add the background to this region
                        "name": "background",
                        "data": background_x.tolist(),
                        # Add the uncertainity to the background
                        "modifiers": [
                            {"name": "bkg_uncert", "type": "shapesys", "data": model_x_errors.tolist()}
                        ]
                    }
                ]
            }
        ]
    }
    

    #Then we make the full json which has the data
    workspace_spec = {
        "version": "1.0.0",
        "channels": model_spec["channels"], #Add in the precictions from above 
        "observations": [
            {"name": "region", "data": data_x.tolist()} #Add in the data
        ],
        "measurements": [ #This is to tell pyhf we will want to fit the 
            {
                "name": "measurement", 
                "config": {
                    "poi": "mu",
                    "parameters": []
                }
            }
        ]
    }
    return workspace_spec

Okay now let's tell pyhf to make the model+data (also called a "workspace" in jargon)

In [ ]:
# Create the workspace object
workspace_spec=makePyhfConfig(data_x,signal_x,background_x,model_x_errors)
ws = pyhf.Workspace(workspace_spec)

model = ws.model()      # Extract just the likelihood model from the workspace
data = ws.data(model)   # Extract just the data from the workspace

We can print out some of the model info to get a feel of things

In [ ]:
print(f"  POI name:   {model.config.poi_name}")
print(f"  channels:   {model.config.channels}")
print(f"     nbins:   {model.config.channel_nbins}")
print(f"   samples:   {model.config.samples}")
print(f" modifiers:   {model.config.modifiers}")
print(f"N paramaters: {model.config.npars}")
print(f"parameters:   {model.config.parameters}")
print(f"  nauxdata:   {model.config.nauxdata}")
#print(f"   auxdata:   {model.config.auxdata}"

Note in this model we have actually specified 61 paramaters: The single poi $\mu$, and 60 independent background uncertainities for each of the 60 bins. These 60 also have matching auxillary data, aka the bins of the background uncertainity histogram we provided, which help the model know how far these paramaters can vary (see the lecture).

Using `cabinetry` we can also make a quick plot of things. Unsuprisingly they look similar to before

In [ ]:
model_prefit = cabinetry.model_utils.prediction(model)
_ = cabinetry.tabulate.yields(model_prefit, data)
_ = cabinetry.visualize.data_mc(model_prefit, data)

Now let's get to the fun stuff! We can now fit this model to the data, and extract our signal strength

In [ ]:
fit_results = cabinetry.fit.fit(model, data)

mu_hat = fit_results.bestfit[model.config.poi_index]
mu_err = fit_results.uncertainty[model.config.poi_index]
print(f"Best-fit mu: {mu_hat} +/- {mu_err}")

Pretty simple isn't it. While there is lots of details in statistsica, the actual implementation of the methods is mostly automatic. The biggest point is specifying a good likelihood model in the first place (and the form of that model)

We can also run some hypothesis testing! It's just as easy. Let's calculate the background-only p-value, aka the "signficance" of the signal. As a reminder in high-energy physics we have the very high-bar that we only reject the background-only model when we have greater then a $5\sigma$ significance

In [ ]:
significance_results = cabinetry.fit.significance(model, data)
print(f" Signficance={significance_results.observed_significance}")

Looks like even with this partial open-dataset we can be very confident that you need the Higgs signal to match the data! (We may have taken some short-cuts in evluating all the necessary sources of systematics, but you get the picture)

But this bring up another point, how confident are we that the Higgs signal is at 125GeV. Why not another mass point? Are we robust to this assumption?!

Let's re-run the signal/background template evaluation from the first step now assuming a mass peak at 140 GeV

In [ ]:
signal_x, background_x, model_x_errors, result =fitModel(bin_centres, data_x, mass=140)
plotAnalysis(bin_centres, data_x, data_x_errors, background_x, signal_x, model_x_errors)

Hmm, doesn't seem likely. Let's re-run the true hypothesis test though to double-check

In [ ]:
workspace_spec=makePyhfConfig(data_x,signal_x,background_x,model_x_errors)
ws = pyhf.Workspace(workspace_spec)
significance_results = cabinetry.fit.significance(ws.model() , ws.data(ws.model()))
print(f" Signficance={significance_results.observed_significance}")

Indeed there is no significant value, and say the data is consistent with a background-only model.

So what masses could be compatible. Let's run a scan! 

In [ ]:
masses=np.linspace(110,140,25)
sigs=[]
for mass in masses:
    signal_x, background_x, model_x_errors,_=fitModel(bin_centres, data_x, mass=mass)
    workspace_spec=makePyhfConfig(data_x,signal_x,background_x,model_x_errors)

    # Create the workspace object
    ws = pyhf.Workspace(workspace_spec)
    model = ws.model()       # same as before, created from workspace
    data = ws.data(model)   # observed data + auxdata automatically assembled
    
    significance_results = cabinetry.fit.significance(model, data)
    print(f" Mass={mass}, Signficance={significance_results.observed_significance}")
    sigs.append(significance_results.observed_significance)

In [ ]:
fig,ax=plt.subplots()
ax.plot(masses,sigs)
ax.set_ylabel('Signal Significance')
ax.set_xlabel('Signal mass [GeV]')

You can compare this to the plot from the Higgs discovery paper which used less data, but more experimental channels https://doi.org/10.1016/j.physletb.2012.08.020 (Just mirror the y-axis)

You can see that the data is favoring a mass at 125 GeV

![](https://cds.cern.ch/record/1471031/files/disc3_0_sig_p0_zoom.png)